In [ ]:
import numpy as np
import pandas as pd
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.sparse import csr_matrix
from weighting_strategies import (
    bm25_weight, tfidf_weight, normalized_weight,
    log_weight, log_idf_weight, power_weight,
    pmi_weight, robust_user_centric_weight, sigmoid_propensity_weight, power_lift_weight, robust_user_centric_weight_v2
)
# We keep implicit evaluation for symmetry and metric consistency with the previous notebook
from implicit.evaluation import train_test_split, ranking_metrics_at_k
import cornac

from autorec import AutoRec

In [4]:
import sys
import os

# Add the parent directory to sys.path to resolve imports from sibling directories
sys.path.append(os.path.abspath(".."))

from utils.sparse import transform_dataframe_to_sparse

In [5]:
movielens_df = (
    pd.DataFrame(
        data=cornac.datasets.movielens.load_feedback(variant="100K"),
        columns=['user_id', 'item_id', 'target']
    )
    .loc[:, ['user_id', 'item_id', 'target']]
    .dropna()
)
movielens_df['user_id'].nunique(), movielens_df['item_id'].nunique(), movielens_df.shape[0]

(943, 1682, 100000)

In [6]:
user_item_matrix, user_mapping, item_mapping = transform_dataframe_to_sparse(
    movielens_df, row_field='user_id', col_field='item_id', data_field='target'
)


train_val_mat, test_mat = train_test_split(user_item_matrix, train_percentage=0.9, random_state=42)
train_mat, val_mat = train_test_split(train_val_mat, train_percentage=0.9, random_state=42)

print(f"Train Shape: {train_mat.shape}, Val Shape: {val_mat.shape}, Test Shape: {test_mat.shape}")

Train Shape: (943, 1682), Val Shape: (943, 1682), Test Shape: (943, 1682)


In [ ]:
results_folder = "results/movielens_100k_autorec"
results_filename = "movielens_100k_autorec_results.csv"
import time

def run_hyperparameter_optimization(
    train_mat: csr_matrix,
    val_mat: csr_matrix,
    train_val_mat: csr_matrix,
    test_mat: csr_matrix,
    weighting_strategy: str,
    algorithm: str,
    n_trials: int = 20,
    output_dir: str = None,
) -> pd.DataFrame:
    results = []
    
    algorithms = {
        "I-AutoRec": lambda: AutoRec(unobserved_weight=0.1, hidden_dim=10, epochs=20),
    }
    strategies = [
        "no_weighting",
        "bm25",
        "tfidf",
        "log", 
        "log_idf
        "power",
        "normalized",
        "pmi",
        "robust_user_centric",
        "robust_user_centric_weight_v2",
        "sigmoid_propensity",
        "power_lift"
    ]
    if weighting_strategy not in strategies:
        raise ValueError(f"Weighting strategy '{weighting_strategy}' is not recognized.")
    strategy = weighting_strategy

    if algorithm not in algorithms:
        raise ValueError(f"Algorithm '{algorithm}' is not recognized.")
    algo_name = algorithm
    AlgoFactory = algorithms[algorithm]

    print(f"Running optimization for {algo_name} with {strategy}...")

    def get_weighted_matrix(matrix, params):
        weighted = matrix.copy()
        if strategy == "bm25":
            weighted = bm25_weight(weighted, K1=params.get("bm25_k1"), B=params.get("bm25_b"))
        elif strategy == "log_idf
            weighted = log_idf_weight(weighted, alpha=params.get("conf_alpha"))
        elif strategy == "power":
            weighted = power_weight(weighted, p=params.get("power_p"))
        elif strategy == "tfidf":
            weighted = tfidf_weight(weighted)
        elif strategy == "log":
            weighted = log_weight(weighted)
        elif strategy == "normalized":
            weighted = normalized_weight(weighted)
        elif strategy == "pmi":
            weighted = pmi_weight(weighted)
        elif strategy == "robust_user_centric":
            weighted = robust_user_centric_weight(weighted, scale_factor=params.get("scale_factor"))
        elif strategy == "sigmoid_propensity":
            weighted = sigmoid_propensity_weight(weighted, p=params.get("p"), beta=params.get("beta"))
        elif strategy == "power_lift":
            weighted = power_lift_weight(weighted, p=params.get("p"))
        elif strategy == "robust_user_centric_weight_v2":
            weighted = robust_user_centric_weight_v2(weighted, lower_q=params.get("lower_q"), upper_q=params.get("upper_q"))
        return weighted

    def objective(trial):
        params = {}
        # Suggest weighting strategy parameters
        if strategy == "bm25":
            params["bm25_k1"] = trial.suggest_float("bm25_k1", 0.1, 1000)
            params["bm25_b"] = trial.suggest_float("bm25_b", 0.0, 1.0)
        elif strategy == "log_idf
            params["conf_alpha"] = trial.suggest_float("conf_alpha", 1.0, 150.0)
        elif strategy == "power":
            params["power_p"] = trial.suggest_float("power_p", 0.1, 1.5)
        elif strategy == "robust_user_centric":
            params["scale_factor"] = trial.suggest_float("scale_factor", 0.1, 10.0)
        elif strategy == "robust_user_centric_weight_v2":
            params["lower_q"] = trial.suggest_float("lower_q", 5.0, 45.0)
            params["upper_q"] = trial.suggest_float("upper_q", 55.0, 95.0)
        elif strategy == "sigmoid_propensity":
            params["p"] = trial.suggest_float("p", 0.1, 5.0)
            params["beta"] = trial.suggest_float("beta", 0.0, 1.0)
        elif strategy == "power_lift":
            params["p"] = trial.suggest_float("p", 0.1, 1.5)
        
        # Apply weighting
        weighted_train = get_weighted_matrix(train_mat, params)
        
        # Train Model
        model = AlgoFactory()
        model.fit(weighted_train, show_progress=False)

        # Evaluate on Validation Set
        return ranking_metrics_at_k(model, train_mat, val_mat, K=20, show_progress=False)['ndcg']

    # Optimize only if strategy has parameters
    current_trials = n_trials if strategy in ["bm25", "log_idf"power", "robust_user_centric", "robust_user_centric_weight_v2", "sigmoid_propensity", "power_lift"] else 1
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=current_trials, n_jobs=-1)

    # --- Final Retraining & Testing ---
    # Use best params to weight the full train_val matrix
    best_params = study.best_params
    weighted_train_val = get_weighted_matrix(train_val_mat, best_params)

    # Train Final Model
    final_model = AlgoFactory()
    
    start_time = time.time()
    final_model.fit(weighted_train_val, show_progress=False)
    end_time = time.time()
    
    # Evaluate on Test Set
    metrics_at_10 = ranking_metrics_at_k(final_model, train_val_mat, test_mat, K=10, show_progress=False)
    metrics_at_20 = ranking_metrics_at_k(final_model, train_val_mat, test_mat, K=20, show_progress=False)

    results.append({
        "Algorithm": algo_name,
        "Strategy": strategy,
        "Number of Optimization Trials": current_trials,
        "Best Val NDCG@20": study.best_value,
        "Test NDCG@10": metrics_at_10['ndcg'],
        "Test NDCG@20": metrics_at_20['ndcg'],
        "Test Precision@10": metrics_at_10['precision'],
        "Test Precision@20": metrics_at_20['precision'],
        "Final Train Time (s)": end_time - start_time,
        "Best Params": best_params
    })

    if output_dir:
        output_path = os.path.join(output_dir, f"{algo_name}_{strategy}_results.csv")
        pd.DataFrame(results).to_csv(output_path, index=False)
    return pd.DataFrame(results)

In [17]:
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

# Execution of all experiments
# Note: Reduced n_trials to 15 for Deep Learning models to save time, or keep at 20.
TRIALS = 15

In [18]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="I-AutoRec", n_trials=TRIALS, output_dir=results_folder)

[I 2026-02-08 00:36:21,350] A new study created in memory with name: no-name-577008e1-6502-4b56-b302-13e9b6c0b864


Running optimization for I-AutoRec with no_weighting...


[I 2026-02-08 00:36:21,901] Trial 0 finished with value: 0.10428389951083251 and parameters: {}. Best is trial 0 with value: 0.10428389951083251.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,no_weighting,1,0.104284,0.084036,0.09285,0.105393,0.133941,0.446734,{}


In [19]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="I-AutoRec", n_trials=TRIALS, output_dir=results_folder)

[I 2026-02-08 00:36:25,632] A new study created in memory with name: no-name-6452ee58-fe9b-4e97-b334-3405d5c38310


Running optimization for I-AutoRec with no_weighting...


[I 2026-02-08 00:36:26,192] Trial 0 finished with value: 0.08720317446522359 and parameters: {}. Best is trial 0 with value: 0.08720317446522359.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,no_weighting,1,0.087203,0.082827,0.093957,0.099621,0.127309,0.442271,{}


In [20]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="I-AutoRec", n_trials=TRIALS, output_dir=results_folder)


[I 2026-02-08 00:36:27,742] A new study created in memory with name: no-name-c30ee7f3-72e2-4057-b761-bd3ebdd84d48


Running optimization for I-AutoRec with bm25...


[I 2026-02-08 00:36:28,301] Trial 0 finished with value: 0.08355089092724195 and parameters: {'bm25_k1': 552.9979706216101, 'bm25_b': 0.6480383749336704}. Best is trial 0 with value: 0.08355089092724195.
[I 2026-02-08 00:36:28,861] Trial 1 finished with value: 0.07792503085038449 and parameters: {'bm25_k1': 953.7334491579076, 'bm25_b': 0.2606446538982895}. Best is trial 0 with value: 0.08355089092724195.
[I 2026-02-08 00:36:29,432] Trial 2 finished with value: 0.07022830344612066 and parameters: {'bm25_k1': 985.487169124622, 'bm25_b': 0.521729951846484}. Best is trial 0 with value: 0.08355089092724195.
[I 2026-02-08 00:36:29,980] Trial 3 finished with value: 0.07178411438544031 and parameters: {'bm25_k1': 836.0950669842033, 'bm25_b': 0.10289226539116225}. Best is trial 0 with value: 0.08355089092724195.
[I 2026-02-08 00:36:30,533] Trial 4 finished with value: 0.07746128645571147 and parameters: {'bm25_k1': 514.7853222479705, 'bm25_b': 0.5183507759984968}. Best is trial 0 with value: 0.

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,bm25,15,0.093054,0.080426,0.092083,0.095662,0.120677,0.450598,"{'bm25_k1': 33.16756390643815, 'bm25_b': 0.322..."


In [21]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="I-AutoRec", n_trials=TRIALS, output_dir=results_folder)

[I 2026-02-08 00:36:37,003] A new study created in memory with name: no-name-3a10c5a4-c003-4675-8fe2-9ab7329aabb4


Running optimization for I-AutoRec with tfidf...


[I 2026-02-08 00:36:37,567] Trial 0 finished with value: 0.07061708652024379 and parameters: {}. Best is trial 0 with value: 0.07061708652024379.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,tfidf,1,0.070617,0.072306,0.078294,0.094508,0.11369,0.47405,{}


In [22]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="I-AutoRec", n_trials=TRIALS, output_dir=results_folder)

[I 2026-02-08 00:36:38,483] A new study created in memory with name: no-name-8f8b0915-3095-41da-8930-c06c9255fdae


Running optimization for I-AutoRec with log...


[I 2026-02-08 00:36:39,038] Trial 0 finished with value: 0.08813979159824074 and parameters: {}. Best is trial 0 with value: 0.08813979159824074.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,log,1,0.08814,0.094266,0.105044,0.116444,0.143297,0.439535,{}


In [ ]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log_idf", algorithm="I-AutoRec", n_trials=TRIALS, output_dir=results_folder)


[I 2026-02-08 00:36:39,916] A new study created in memory with name: no-name-55ab9f57-540e-4f9f-b5f2-c53aa96681f6


Running optimization for I-AutoRec with confidence...


[I 2026-02-08 00:36:40,475] Trial 0 finished with value: 0.08643335547396985 and parameters: {'conf_alpha': 16.041883473467085}. Best is trial 0 with value: 0.08643335547396985.
[I 2026-02-08 00:36:41,054] Trial 1 finished with value: 0.08876316676714713 and parameters: {'conf_alpha': 29.63565086438437}. Best is trial 1 with value: 0.08876316676714713.
[I 2026-02-08 00:36:41,629] Trial 2 finished with value: 0.08069738958813641 and parameters: {'conf_alpha': 107.32289016299967}. Best is trial 1 with value: 0.08876316676714713.
[I 2026-02-08 00:36:42,227] Trial 3 finished with value: 0.08003150688628653 and parameters: {'conf_alpha': 69.5689332466749}. Best is trial 1 with value: 0.08876316676714713.
[I 2026-02-08 00:36:42,975] Trial 4 finished with value: 0.09492890724175183 and parameters: {'conf_alpha': 74.69247901504902}. Best is trial 4 with value: 0.09492890724175183.
[I 2026-02-08 00:36:43,544] Trial 5 finished with value: 0.09357924998186963 and parameters: {'conf_alpha': 130.69

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,confidence,15,0.094929,0.080074,0.091874,0.094013,0.119493,0.443505,{'conf_alpha': 74.69247901504902}


In [24]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="I-AutoRec", n_trials=TRIALS, output_dir=results_folder)


[I 2026-02-08 00:36:49,516] A new study created in memory with name: no-name-9ab88199-ba54-49d5-ac18-c4620fba104b


Running optimization for I-AutoRec with power...


[I 2026-02-08 00:36:50,068] Trial 0 finished with value: 0.07610374507983457 and parameters: {'power_p': 0.8275271711968825}. Best is trial 0 with value: 0.07610374507983457.
[I 2026-02-08 00:36:50,655] Trial 1 finished with value: 0.08151342758033975 and parameters: {'power_p': 1.4595891498674982}. Best is trial 1 with value: 0.08151342758033975.
[I 2026-02-08 00:36:51,249] Trial 2 finished with value: 0.07361714458551982 and parameters: {'power_p': 0.25412814982820253}. Best is trial 1 with value: 0.08151342758033975.
[I 2026-02-08 00:36:51,794] Trial 3 finished with value: 0.09065826075978375 and parameters: {'power_p': 1.3839376921555946}. Best is trial 3 with value: 0.09065826075978375.
[I 2026-02-08 00:36:52,342] Trial 4 finished with value: 0.0758730879406797 and parameters: {'power_p': 1.2110897305737087}. Best is trial 3 with value: 0.09065826075978375.
[I 2026-02-08 00:36:52,892] Trial 5 finished with value: 0.08548613822492182 and parameters: {'power_p': 1.0663647645009946}.

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,power,15,0.093604,0.080803,0.092449,0.102095,0.134415,0.451363,{'power_p': 0.8902548200805446}


In [25]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="I-AutoRec", n_trials=TRIALS, output_dir=results_folder)


[I 2026-02-08 00:36:58,664] A new study created in memory with name: no-name-848eb9f0-cd8b-4285-b017-f07e049ffc0e


Running optimization for I-AutoRec with normalized...


[I 2026-02-08 00:36:59,221] Trial 0 finished with value: 0.05248731288361221 and parameters: {}. Best is trial 0 with value: 0.05248731288361221.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,normalized,1,0.052487,0.046292,0.055194,0.058057,0.077333,0.447264,{}


In [26]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="I-AutoRec", n_trials=TRIALS, output_dir=results_folder)


[I 2026-02-08 00:37:00,104] A new study created in memory with name: no-name-0b7ce0f2-f42a-4146-b3b8-2a98849a2555


Running optimization for I-AutoRec with pmi...


[I 2026-02-08 00:37:00,664] Trial 0 finished with value: 0.07481377787742423 and parameters: {}. Best is trial 0 with value: 0.07481377787742423.


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,pmi,1,0.074814,0.079151,0.090247,0.093353,0.119493,0.449346,{}


In [27]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="I-AutoRec", n_trials=TRIALS, output_dir=results_folder)


[I 2026-02-08 00:37:01,549] A new study created in memory with name: no-name-4cb13eca-af1e-40d4-9f6e-0462bfde5046


Running optimization for I-AutoRec with robust_user_centric...


[I 2026-02-08 00:37:02,122] Trial 0 finished with value: 0.08486451612709854 and parameters: {'scale_factor': 2.0218273629298475}. Best is trial 0 with value: 0.08486451612709854.
[I 2026-02-08 00:37:02,676] Trial 1 finished with value: 0.09117054428264854 and parameters: {'scale_factor': 5.6701968544903485}. Best is trial 1 with value: 0.09117054428264854.
[I 2026-02-08 00:37:03,230] Trial 2 finished with value: 0.08095984605500428 and parameters: {'scale_factor': 5.765329184433719}. Best is trial 1 with value: 0.09117054428264854.
[I 2026-02-08 00:37:03,784] Trial 3 finished with value: 0.08496124391339673 and parameters: {'scale_factor': 3.00693292083462}. Best is trial 1 with value: 0.09117054428264854.
[I 2026-02-08 00:37:04,335] Trial 4 finished with value: 0.09153072582838763 and parameters: {'scale_factor': 3.2850540834474997}. Best is trial 4 with value: 0.09153072582838763.
[I 2026-02-08 00:37:04,899] Trial 5 finished with value: 0.07723358826228795 and parameters: {'scale_fa

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,robust_user_centric,15,0.097479,0.09222,0.102221,0.107702,0.135007,0.445324,{'scale_factor': 6.837650716984776}


In [28]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="I-AutoRec", n_trials=TRIALS, output_dir=results_folder)


[I 2026-02-08 00:37:10,805] A new study created in memory with name: no-name-6f0fc109-339b-450a-8161-a6a0f7caadba


Running optimization for I-AutoRec with robust_user_centric_weight_v2...


[I 2026-02-08 00:37:11,358] Trial 0 finished with value: 0.07725497451654928 and parameters: {'lower_q': 40.406970209967156, 'upper_q': 93.3059402082664}. Best is trial 0 with value: 0.07725497451654928.
[I 2026-02-08 00:37:11,910] Trial 1 finished with value: 0.07719729340497229 and parameters: {'lower_q': 12.294933121135303, 'upper_q': 78.17795841797826}. Best is trial 0 with value: 0.07725497451654928.
[I 2026-02-08 00:37:12,467] Trial 2 finished with value: 0.08617033884952374 and parameters: {'lower_q': 16.474289092286227, 'upper_q': 57.57033520460195}. Best is trial 2 with value: 0.08617033884952374.
[I 2026-02-08 00:37:13,022] Trial 3 finished with value: 0.0831001027598318 and parameters: {'lower_q': 25.027196041848686, 'upper_q': 88.92458381693216}. Best is trial 2 with value: 0.08617033884952374.
[I 2026-02-08 00:37:13,571] Trial 4 finished with value: 0.08555026352671209 and parameters: {'lower_q': 43.81791617455362, 'upper_q': 74.03737348707818}. Best is trial 2 with value:

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,robust_user_centric_weight_v2,15,0.086568,0.087677,0.096675,0.109187,0.137731,0.446095,"{'lower_q': 10.482242679024125, 'upper_q': 64...."


In [29]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="I-AutoRec", n_trials=TRIALS, output_dir=results_folder)


[I 2026-02-08 00:37:20,237] A new study created in memory with name: no-name-79adce79-9bf7-45b7-be42-55ab15e49d0a


Running optimization for I-AutoRec with sigmoid_propensity...


[I 2026-02-08 00:37:20,791] Trial 0 finished with value: 0.08005869780007561 and parameters: {'p': 2.7502857515831654, 'beta': 0.3265611726644163}. Best is trial 0 with value: 0.08005869780007561.
[I 2026-02-08 00:37:21,342] Trial 1 finished with value: 0.07568942403836562 and parameters: {'p': 4.112486343928838, 'beta': 0.2651006976895365}. Best is trial 0 with value: 0.08005869780007561.
[I 2026-02-08 00:37:21,978] Trial 2 finished with value: 0.08011596704281311 and parameters: {'p': 4.82425710062541, 'beta': 0.7107985787298889}. Best is trial 2 with value: 0.08011596704281311.
[I 2026-02-08 00:37:22,561] Trial 3 finished with value: 0.08265277554051347 and parameters: {'p': 2.6058005296808147, 'beta': 0.6292737354566252}. Best is trial 3 with value: 0.08265277554051347.
[I 2026-02-08 00:37:23,243] Trial 4 finished with value: 0.08908169910862292 and parameters: {'p': 2.7461130372325364, 'beta': 0.2858097275664312}. Best is trial 4 with value: 0.08908169910862292.
[I 2026-02-08 00:3

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,sigmoid_propensity,15,0.089082,0.081431,0.089594,0.097476,0.121151,0.443627,"{'p': 2.7461130372325364, 'beta': 0.2858097275..."


In [30]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="I-AutoRec", n_trials=TRIALS, output_dir=results_folder)

[I 2026-02-08 00:37:29,654] A new study created in memory with name: no-name-17b66599-9ba5-483c-a675-b2b842f050c1


Running optimization for I-AutoRec with power_lift...


[I 2026-02-08 00:37:30,214] Trial 0 finished with value: 0.08369444367489304 and parameters: {'p': 0.9135165804318888}. Best is trial 0 with value: 0.08369444367489304.
[I 2026-02-08 00:37:30,770] Trial 1 finished with value: 0.07125525383308429 and parameters: {'p': 1.3551558466693292}. Best is trial 0 with value: 0.08369444367489304.
[I 2026-02-08 00:37:31,327] Trial 2 finished with value: 0.07197898421787102 and parameters: {'p': 0.6793078531751457}. Best is trial 0 with value: 0.08369444367489304.
[I 2026-02-08 00:37:31,881] Trial 3 finished with value: 0.08293566035363095 and parameters: {'p': 1.0821574819127355}. Best is trial 0 with value: 0.08369444367489304.
[I 2026-02-08 00:37:32,431] Trial 4 finished with value: 0.0767140755307511 and parameters: {'p': 1.4338348657152842}. Best is trial 0 with value: 0.08369444367489304.
[I 2026-02-08 00:37:32,978] Trial 5 finished with value: 0.0755549425146015 and parameters: {'p': 0.34286587748405334}. Best is trial 0 with value: 0.083694

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,power_lift,15,0.087947,0.086401,0.097813,0.106053,0.139981,0.448097,{'p': 0.14181126603068628}


In [32]:
import glob

all_results = []
# Match any CSV in the result folder
for f in glob.glob(f"{results_folder}/*.csv"):
    all_results.append(pd.read_csv(f))

if all_results:
    experiment_results = pd.concat(all_results)
    experiment_results = experiment_results.sort_values("Test NDCG@20", ascending=False)
    experiment_results.to_csv(results_filename, index=False)
else:
    print("No results found.")

experiment_results

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,I-AutoRec,log,1,0.088140,0.094266,0.105044,0.116444,0.143297,0.439535,{}
0,I-AutoRec,robust_user_centric,15,0.097479,0.092220,0.102221,0.107702,0.135007,0.445324,{'scale_factor': 6.837650716984776}
0,I-AutoRec,power_lift,15,0.087947,0.086401,0.097813,0.106053,0.139981,0.448097,{'p': 0.14181126603068628}
0,I-AutoRec,robust_user_centric_weight_v2,15,0.086568,0.087677,0.096675,0.109187,0.137731,0.446095,"{'lower_q': 10.482242679024125, 'upper_q': 64...."
0,I-AutoRec,no_weighting,1,0.087203,0.082827,0.093957,0.099621,0.127309,0.442271,{}
0,I-AutoRec,power,15,0.093604,0.080803,0.092449,0.102095,0.134415,0.451363,{'power_p': 0.8902548200805446}
0,I-AutoRec,bm25,15,0.093054,0.080426,0.092083,0.095662,0.120677,0.450598,"{'bm25_k1': 33.16756390643815, 'bm25_b': 0.322..."
0,I-AutoRec,confidence,15,0.094929,0.080074,0.091874,0.094013,0.119493,0.443505,{'conf_alpha': 74.69247901504902}
0,I-AutoRec,pmi,1,0.074814,0.079151,0.090247,0.093353,0.119493,0.449346,{}
0,I-AutoRec,sigmoid_propensity,15,0.089082,0.081431,0.089594,0.097476,0.121151,0.443627,"{'p': 2.7461130372325364, 'beta': 0.2858097275..."
